In [ ]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

data = {}

with open('tetris_train.json') as json_file:
    data['train'] = json.load(json_file)

with open('tetris_test.json') as json_file:
    data['test'] = json.load(json_file)

labels = {'S': 0, 'T': 1, 'L': 2,'O': 3,'I': 4}

X_train, Y_train, X_test, Y_test = [],[],[],[]

for key in data['train'].keys():
    X_train.extend(data['train'][key])
    for _ in range(len(data['train'][key])):
        Y_train.append(labels[key])

for key in data['test'].keys():
    X_test.extend(data['test'][key])
    for _ in range(len(data['test'][key])):
        Y_test.append(labels[key])

X_train = torch.FloatTensor(X_train)
Y_train = torch.FloatTensor(Y_train)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)



# print(X_train.size()); print(Y_train.size()); print(X_test.size()); print(Y_test.size())

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 2, padding=0)
        self.mp1 = nn.MaxPool2d(2, stride=1, padding=1)
        self.conv2 = nn.Conv2d(2, 6, 2, padding=0)
#         self.mp2 = nn.MaxPool2d(3, stride=1, padding=1)
        self.fc = nn.Linear(24, 5)


    def forward(self, x):
        in_size = x.size(0)
        x = self.mp1(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

model = ConvNet()

# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.5, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.01)


def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

print(get_n_params(ConvNet()))

# 
# Training
# 
results = {} #{iter: [loss, accuracy]}
for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    dataset = TensorDataset(X_train, Y_train)
    train_loader = DataLoader(dataset=dataset,
                          batch_size=200,
                          shuffle=True,
                          num_workers=2)
    
    for _, data in enumerate(train_loader, 1):
#     for i in range(len(Y_train)):
        # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = X_train[i], torch.tensor(Y_train[i])
        inputs, labels = data
#         inputs = inputs.view(256, 1, 3, 3)
#         for i in range(len(labels)):
#             x = inputs[i].view(1, 1, 3, 3)

        inputs = inputs.view(200,1,3,3)
        labels = labels.long()
        optimizer.zero_grad()
        outputs = model(inputs)
#         print(outputs)
#         print(labels[i])
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if epoch % 10 == 9:
        print ("Epoch: %d, Loss: %.3f"%(epoch+1, running_loss))

        correct = 0.
        total = float(len(Y_test))

        with torch.no_grad():
            for i in range(len(Y_test)):
                images, labels = X_test[i], Y_test[i]
                images = images.view(1, 1, 3, 3)
#                 labels = labels.squeeze(0).max(0)[1]
                output = model(images)
                val, predicted = output.data.squeeze(0).max(0)
                correct += int(predicted.float()==labels.float())
                accuracy = 100. * correct / total
        print('Accuracy of the network: %.2f %%' % (accuracy))

        results[epoch+1] = [running_loss, accuracy]

In [ ]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

data = {}

with open('tetris_train.json') as json_file:
    data['train'] = json.load(json_file)

with open('tetris_test.json') as json_file:
    data['test'] = json.load(json_file)

labels = {
    'S': [0,0,0,0,1],
    'T': [0,0,0,1,0],
    'L': [0,0,1,0,0],
    'O': [0,1,0,0,0],
    'I': [1,0,0,0,0]
}

X_train, Y_train, X_test, Y_test = [],[],[],[]

for key in data['train'].keys():
    X_train.extend(data['train'][key])
    for _ in range(len(data['train'][key])):
        Y_train.append(labels[key])

for key in data['test'].keys():
    X_test.extend(data['test'][key])
    for _ in range(len(data['test'][key])):
        Y_test.append(labels[key])


X_train = torch.FloatTensor(X_train)
Y_train = torch.FloatTensor(Y_train)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)

# print(X_train.size()); print(Y_train.size()); print(X_test.size()); print(Y_test.size())

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 2, padding=0)
        self.mp1 = nn.MaxPool2d(2, stride=1)
#         self.conv2 = nn.Conv2d(3, 6, 2, padding=0)
#         self.mp2 = nn.MaxPool2d(3, stride=1, padding=1)
        self.fc = nn.Linear(2, 5)        
        
#         self.conv1 = nn.Conv2d(1, 5, 2)
#         self.conv2 = nn.Conv2d(5, 10, 2)
#         self.fc = nn.Linear(10, 5)

    def forward(self, x):
        in_size = x.size(0)
        x = self.mp1(F.relu(self.conv1(x)))
#         x = self.mp2(F.relu(self.conv2(x)))       
#         x = self.conv1(x)
#         x = self.conv2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

model = ConvNet()

criterion = nn.MSELoss()

# optimizer = optim.SGD(model.parameters(), lr=0.5, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.01)


def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

print(get_n_params(ConvNet()))

# Training
results = {} #{iter: [loss, accuracy]}
for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    dataset = TensorDataset(X_train, Y_train)
    train_loader = DataLoader(dataset=dataset,
                          batch_size=200,
                          shuffle=True,
                          num_workers=2)
    
    for _, data in enumerate(train_loader, 0):

        inputs, labels = data

        inputs = inputs.view(200,1,3,3)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if epoch % 10 == 9:
        print ("Epoch: %d, Loss: %.3f"%(epoch+1, running_loss))

        correct = 0.
        total = float(len(Y_test))

        with torch.no_grad():
            for i in range(len(Y_test)):
                images, labels = X_test[i], Y_test[i]
                images = images.view(1, 1, 3, 3)
                labels = labels.max(0)[1]
                output = model(images)
#                 print(output, labels)
                _, predicted = output.data.squeeze(0).max(0)
        
                correct += int(predicted.float()==labels.float())
                accuracy = 100. * correct / total
        print('Accuracy of the network: %.2f %%' % (accuracy))

        results[epoch+1] = [running_loss, accuracy]



In [1]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

data = {}

with open('tetris_train.json') as json_file:
    data['train'] = json.load(json_file)

with open('tetris_test.json') as json_file:
    data['test'] = json.load(json_file)

labels = {
    'S': [0,0,0,0,1],
    'T': [0,0,0,1,0],
    'L': [0,0,1,0,0],
    'O': [0,1,0,0,0],
    'I': [1,0,0,0,0]
}

X_train, Y_train, X_test, Y_test = [],[],[],[]

for key in data['train'].keys():
    X_train.extend(data['train'][key])
    for _ in range(len(data['train'][key])):
        Y_train.append(labels[key])

for key in data['test'].keys():
    X_test.extend(data['test'][key])
    for _ in range(len(data['test'][key])):
        Y_test.append(labels[key])


X_train = torch.FloatTensor(X_train)
Y_train = torch.FloatTensor(Y_train)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)

# print(X_train.size()); print(Y_train.size()); print(X_test.size()); print(Y_test.size())

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 2, padding=0)
        self.mp1 = nn.MaxPool2d(2, stride=1, padding=1)
        self.conv2 = nn.Conv2d(2, 6, 2, padding=0)
        self.fc = nn.Linear(24, 5)        


    def forward(self, x):
        in_size = x.size(0)
        x = self.mp1(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

# Training
results = {} #{iter: [loss, accuracy]}
for _iter in range(10):
    model = ConvNet()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    _loss = []
    _acc = []
    for epoch in range(1000):  # loop over the dataset multiple times

        running_loss = 0.0
        dataset = TensorDataset(X_train, Y_train)
        train_loader = DataLoader(dataset=dataset,
                              batch_size=200,
                              shuffle=True,
                              num_workers=2)

        for _, data in enumerate(train_loader, 0):

            inputs, labels = data

            inputs = inputs.view(200,1,3,3)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

#         if epoch % 10 == 9:
#             print ("Epoch: %d, Loss: %.3f"%(epoch+1, running_loss))

        correct = 0.
        total = float(len(Y_test))

        with torch.no_grad():
            for i in range(len(Y_test)):
                images, labels = X_test[i], Y_test[i]
                images = images.view(1, 1, 3, 3)
                labels = labels.max(0)[1]
                output = model(images)
#                 print(output, labels)
                _, predicted = output.data.squeeze(0).max(0)

                correct += int(predicted.float()==labels.float())
        
        accuracy = 100. * correct / total
        _loss += [running_loss]
        _acc += [accuracy]

    results[_iter] = {'accuracy': _acc, 'loss':_loss}

print('Done')
# print(results)
with open('results/cmcf5.json', 'w') as fp:
    json.dump(results, fp)


Done
{0: {'accuracy': [20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0,

In [2]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

data = {}

with open('tetris_train.json') as json_file:
    data['train'] = json.load(json_file)

with open('tetris_test.json') as json_file:
    data['test'] = json.load(json_file)

labels = {
    'S': [0,0,0,0,1],
    'T': [0,0,0,1,0],
    'L': [0,0,1,0,0],
    'O': [0,1,0,0,0],
    'I': [1,0,0,0,0]
}

X_train, Y_train, X_test, Y_test = [],[],[],[]

for key in data['train'].keys():
    X_train.extend(data['train'][key])
    for _ in range(len(data['train'][key])):
        Y_train.append(labels[key])

for key in data['test'].keys():
    X_test.extend(data['test'][key])
    for _ in range(len(data['test'][key])):
        Y_test.append(labels[key])


X_train = torch.FloatTensor(X_train)
Y_train = torch.FloatTensor(Y_train)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)

# print(X_train.size()); print(Y_train.size()); print(X_test.size()); print(Y_test.size())

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 2, padding=0)
        self.mp1 = nn.MaxPool2d(2, stride=1)
        self.fc = nn.Linear(2, 5)        


    def forward(self, x):
        in_size = x.size(0)
        x = self.mp1(F.relu(self.conv1(x)))
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

# Training
results = {} #{iter: [loss, accuracy]}
for _iter in range(10):
    model = ConvNet()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    _loss = []
    _acc = []
    
    for epoch in range(1000):  # loop over the dataset multiple times

        running_loss = 0.0
        dataset = TensorDataset(X_train, Y_train)
        train_loader = DataLoader(dataset=dataset,
                              batch_size=200,
                              shuffle=True,
                              num_workers=2)

        for _, data in enumerate(train_loader, 0):

            inputs, labels = data

            inputs = inputs.view(200,1,3,3)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

#         if epoch % 10 == 9:
#             print ("Epoch: %d, Loss: %.3f"%(epoch+1, running_loss))

        correct = 0.
        total = float(len(Y_test))

        with torch.no_grad():
            for i in range(len(Y_test)):
                images, labels = X_test[i], Y_test[i]
                images = images.view(1, 1, 3, 3)
                labels = labels.max(0)[1]
                output = model(images)
#                 print(output, labels)
                _, predicted = output.data.squeeze(0).max(0)

                correct += int(predicted.float()==labels.float())
        
        accuracy = 100. * correct / total
        _loss += [running_loss]
        _acc += [accuracy]

    results[_iter] = {'accuracy': _acc, 'loss':_loss}

print('Done')

with open('results/cmf5.json', 'w') as fp:
    json.dump(results, fp)

Done


In [14]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

data = {}

with open('tetris_train.json') as json_file:
    data['train'] = json.load(json_file)

with open('tetris_test.json') as json_file:
    data['test'] = json.load(json_file)

labels = {
    'S': [0,0,0,0,1],
    'O': [0,1,0,0,0],
}

X_train, Y_train, X_test, Y_test = [],[],[],[]

for key in data['train'].keys():
    if key in labels.keys():
        X_train.extend(data['train'][key])
        for _ in range(len(data['train'][key])):
            Y_train.append(labels[key])

for key in data['test'].keys():
    if key in labels.keys():
        X_test.extend(data['test'][key])
        for _ in range(len(data['test'][key])):
            Y_test.append(labels[key])


X_train = torch.FloatTensor(X_train)
Y_train = torch.FloatTensor(Y_train)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)

# print(X_train.size()); print(Y_train.size()); print(X_test.size()); print(Y_test.size())

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 2, padding=0)
        self.mp1 = nn.MaxPool2d(2, stride=1, padding=1)
        self.conv2 = nn.Conv2d(2, 6, 2, padding=0)
        self.fc = nn.Linear(24, 5)        


    def forward(self, x):
        in_size = x.size(0)
        x = self.mp1(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

# Training
results = {} #{iter: [loss, accuracy]}
for _iter in range(10):
    model = ConvNet()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    _loss = []
    _acc = []
    for epoch in range(1000):  # loop over the dataset multiple times

        running_loss = 0.0
        dataset = TensorDataset(X_train, Y_train)
        train_loader = DataLoader(dataset=dataset,
                              batch_size=200,
                              shuffle=True,
                              num_workers=2)

        for _, data in enumerate(train_loader, 0):

            inputs, labels = data

            inputs = inputs.view(inputs.size(0),1,3,3)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

#         if epoch % 10 == 9:
#             print ("Epoch: %d, Loss: %.3f"%(epoch+1, running_loss))

        correct = 0.
        total = float(len(Y_test))

        with torch.no_grad():
            for i in range(len(Y_test)):
                images, labels = X_test[i], Y_test[i]
                images = images.view(1, 1, 3, 3)
                labels = labels.max(0)[1]
                output = model(images)
#                 print(output, labels)
                _, predicted = output.data.squeeze(0).max(0)

                correct += int(predicted.float()==labels.float())
        
        accuracy = 100. * correct / total
        _loss += [running_loss]
        _acc += [accuracy]

    results[_iter] = {'accuracy': _acc, 'loss':_loss}

print('Done')
print(results)
with open('results/cmcf2.json', 'w') as fp:
    json.dump(results, fp)



Done
{0: {'accuracy': [50.0, 50.0, 50.0, 50.0, 56.25, 56.25, 48.75, 38.75, 41.25, 51.25, 55.0, 56.25, 55.0, 55.0, 57.5, 56.25, 52.5, 52.5, 56.25, 60.0, 66.25, 67.5, 68.75, 68.75, 68.75, 67.5, 67.5, 68.75, 68.75, 68.75, 67.5, 67.5, 67.5, 67.5, 67.5, 67.5, 68.75, 68.75, 68.75, 73.75, 73.75, 81.25, 88.75, 90.0, 92.5, 93.75, 95.0, 93.75, 95.0, 92.5, 93.75, 95.0, 95.0, 96.25, 97.5, 97.5, 97.5, 97.5, 98.75, 98.75, 98.75, 98.75, 98.75, 98.75, 98.75, 98.75, 98.75, 98.75, 98.75, 100.0, 98.75, 100.0, 100.0, 98.75, 98.75, 100.0, 100.0, 98.75, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 

In [ ]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

data = {}

with open('tetris_train.json') as json_file:
    data['train'] = json.load(json_file)

with open('tetris_test.json') as json_file:
    data['test'] = json.load(json_file)

labels = {
    'S': [0,0,0,0,1],
#     'T': [0,0,0,1,0],
#     'L': [0,0,1,0,0],
    'O': [0,1,0,0,0],
#     'I': [1,0,0,0,0]
}

X_train, Y_train, X_test, Y_test = [],[],[],[]

for key in data['train'].keys():
    if key in labels.keys():
        X_train.extend(data['train'][key])
        for _ in range(len(data['train'][key])):
            Y_train.append(labels[key])

for key in data['test'].keys():
    if key in labels.keys():
        X_test.extend(data['test'][key])
        for _ in range(len(data['test'][key])):
            Y_test.append(labels[key])


X_train = torch.FloatTensor(X_train)
Y_train = torch.FloatTensor(Y_train)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)

# print(X_train.size()); print(Y_train.size()); print(X_test.size()); print(Y_test.size())

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 2, padding=0)
        self.mp1 = nn.MaxPool2d(2, stride=1)
        self.fc = nn.Linear(2, 5)        


    def forward(self, x):
        in_size = x.size(0)
        x = self.mp1(F.relu(self.conv1(x)))
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

# Training
results = {} #{iter: [loss, accuracy]}
for _iter in range(10):
    model = ConvNet()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    _loss = []
    _acc = []
    
    for epoch in range(1000):  # loop over the dataset multiple times

        running_loss = 0.0
        dataset = TensorDataset(X_train, Y_train)
        train_loader = DataLoader(dataset=dataset,
                              batch_size=200,
                              shuffle=True,
                              num_workers=2)

        for _, data in enumerate(train_loader, 0):

            inputs, labels = data

            inputs = inputs.view(inputs.size(0),1,3,3)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

#         if epoch % 10 == 9:
#             print ("Epoch: %d, Loss: %.3f"%(epoch+1, running_loss))

        correct = 0.
        total = float(len(Y_test))

        with torch.no_grad():
            for i in range(len(Y_test)):
                images, labels = X_test[i], Y_test[i]
                images = images.view(1, 1, 3, 3)
                labels = labels.max(0)[1]
                output = model(images)
#                 print(output, labels)
                _, predicted = output.data.squeeze(0).max(0)

                correct += int(predicted.float()==labels.float())
        
        accuracy = 100. * correct / total
        _loss += [running_loss]
        _acc += [accuracy]

    results[_iter] = {'accuracy': _acc, 'loss':_loss}

print('Done')

with open('results/cmf2.json', 'w') as fp:
    json.dump(results, fp)

In [12]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

data = {}

with open('tetris_train.json') as json_file:
    data['train'] = json.load(json_file)

with open('tetris_test.json') as json_file:
    data['test'] = json.load(json_file)

labels = {
    'S': 0.,
    'O': 1.,
}

X_train, Y_train, X_test, Y_test = [],[],[],[]

for key in data['train'].keys():
    if key in labels.keys():
        X_train.extend(data['train'][key])
        for _ in range(len(data['train'][key])):
            Y_train.append(labels[key])

for key in data['test'].keys():
    if key in labels.keys():
        X_test.extend(data['test'][key])
        for _ in range(len(data['test'][key])):
            Y_test.append(labels[key])


X_train = torch.FloatTensor(X_train)
Y_train = torch.FloatTensor(Y_train)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)


class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 2, padding=0)
        self.mp1 = nn.MaxPool2d(2, stride=1)
        self.fc = nn.Linear(2, 1)        


    def forward(self, x):
        in_size = x.size(0)
        x = self.mp1(F.relu(self.conv1(x)))
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

# Training
results = {} #{iter: [loss, accuracy]}
for _iter in range(10):
    model = ConvNet()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    _loss = []
    _acc = []
    
    for epoch in range(1000):  # loop over the dataset multiple times

        running_loss = 0.0
        dataset = TensorDataset(X_train, Y_train)
        train_loader = DataLoader(dataset=dataset,
                              batch_size=200,
                              shuffle=True,
                              num_workers=2)

        for _, data in enumerate(train_loader, 0):

            inputs, labels = data

            inputs = inputs.view(len(inputs),1,3,3)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        correct = 0.
        total = float(len(Y_test))

        with torch.no_grad():
            for i in range(len(Y_test)):
                images, labels = X_test[i], Y_test[i]
                images = images.view(1, 1, 3, 3)

                output = model(images)
                predicted = output.data > 0.5
                correct += int(predicted.float()==labels.float())
        
        accuracy = 100. * correct / total

        _loss += [running_loss]
        _acc += [accuracy]

    results[_iter] = {'accuracy': _acc, 'loss':_loss}

print('Done')

with open('results/cmf2.json', 'w') as fp:
    json.dump(results, fp)

Done


In [13]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

data = {}

with open('tetris_train.json') as json_file:
    data['train'] = json.load(json_file)

with open('tetris_test.json') as json_file:
    data['test'] = json.load(json_file)

labels = {
    'S': 0.,
    'O': 1.,
}

X_train, Y_train, X_test, Y_test = [],[],[],[]

for key in data['train'].keys():
    if key in labels.keys():
        X_train.extend(data['train'][key])
        for _ in range(len(data['train'][key])):
            Y_train.append(labels[key])

for key in data['test'].keys():
    if key in labels.keys():
        X_test.extend(data['test'][key])
        for _ in range(len(data['test'][key])):
            Y_test.append(labels[key])


X_train = torch.FloatTensor(X_train)
Y_train = torch.FloatTensor(Y_train)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)


class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 2, padding=0)
        self.mp1 = nn.MaxPool2d(2, stride=1, padding=1)
        self.conv2 = nn.Conv2d(2, 6, 2, padding=0)
        self.fc = nn.Linear(24, 1)        


    def forward(self, x):
        in_size = x.size(0)
        x = self.mp1(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

# Training
results = {} #{iter: [loss, accuracy]}
for _iter in range(10):
    model = ConvNet()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    _loss = []
    _acc = []
    
    for epoch in range(1000):  # loop over the dataset multiple times

        running_loss = 0.0
        dataset = TensorDataset(X_train, Y_train)
        train_loader = DataLoader(dataset=dataset,
                              batch_size=200,
                              shuffle=True,
                              num_workers=2)

        for _, data in enumerate(train_loader, 0):

            inputs, labels = data

            inputs = inputs.view(len(inputs),1,3,3)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        correct = 0.
        total = float(len(Y_test))

        with torch.no_grad():
            for i in range(len(Y_test)):
                images, labels = X_test[i], Y_test[i]
                images = images.view(1, 1, 3, 3)

                output = model(images)
                predicted = output.data > 0.5
                correct += int(predicted.float()==labels.float())
        
        accuracy = 100. * correct / total

        _loss += [running_loss]
        _acc += [accuracy]

    results[_iter] = {'accuracy': _acc, 'loss':_loss}

print('Done')

with open('results/cmcf2.json', 'w') as fp:
    json.dump(results, fp)

Done
